<a href="https://colab.research.google.com/github/MuhammadHelmyOmar/ArabicPIIRedaction/blob/main/fine-tuning_SLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [9]:
import pandas as pd

### Test data
- Original Competition Data

In [10]:
data_url = "https://raw.githubusercontent.com/MuhammadHelmyOmar/ArabicPIIRedaction/main/data/data_final.xlsx"

test = pd.read_excel(data_url)

test.head()

,source,target,dialect
0,الأستاذ أدولفوس ريغان زييمان، بصفتك المدير الت...,الأستاذ [MASK]، بصفتك المدير التنفيذي الرئيسي ...,EGYPT
1,مرحبًا يا هانا، ممكن لو سمحتي تتحققي من التداع...,مرحبًا يا [MASK]، ممكن لو سمحتي تتحققي من التد...,EGYPT
2,نطلب أيضًا مراجعة سياساتنا فيما يتعلق بالتغيير...,نطلب أيضًا مراجعة سياساتنا فيما يتعلق بالتغيير...,SAUDI
3,عزيزي ديفان، مطلوب تقديم عرض تقديمي على مستوى ...,عزيزي [MASK]، مطلوب تقديم عرض تقديمي على مستوى...,EGYPT
4,ممكن كمان نعمل جلسة عن كيفية إدارة التوتر والت...,The sentence doesn't contain any personal or s...,LEVANTINE


### Train data
- Created by Mosaab

In [21]:
from google.colab import drive
import random

In [ ]:
drive.mount('/content/drive')

In [18]:
data_path = "/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/ar_dataset_translated_fixed.csv"

train = pd.read_csv(data_path)

In [19]:
train.drop(["Unnamed: 0",'lang'], axis=1, inplace=True)
print(len(train))
train.head()

209261


,source,tokens,tags,clean_source
0,مرحبًا <!FIRSTNAME_1!> Johan </!FIRSTNAME_1!> ...,"['مرحبا', 'Johan', 'Walker', '،', 'إشعار', 'ال...","['O', 'B-FIRSTNAME', 'B-LASTNAME', 'O', 'O', '...",مرحبا Johan Walker، إشعار الامتثال: تتطلب التغ...
1,نحن نقوم بمراجعة القضية التي رفعها <!FIRSTNAME...,"['نحن', 'نقوم', 'بمراجعة', 'القضية', 'التي', '...","['O', 'O', 'O', 'O', 'O', 'O', 'B-FIRSTNAME', ...",نحن نقوم بمراجعة القضية التي رفعها كادي شوبي ب...
2,بسبب اتفاقية السرية، لا يمكن للمريضة <!SEX_1!>...,"['بسبب', 'اتفاقية', 'السرية', '،', 'لا', 'يمكن...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEX', '...",بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...
3,عزيزي ولي الأمر، يلزم الحصول على إذن في <!DATE...,"['عزيزي', 'ولي', 'الأمر', '،', 'يلزم', 'الحصول...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...
4,تنظيم اجتماع حول <!AGE_1!> 7 سنوات </!AGE_1!> ...,"['تنظيم', 'اجتماع', 'حول', '7', 'سنوات', '(', ...","['O', 'O', 'O', 'B-AGE', 'I-AGE', 'O', 'B-GEND...",تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...


In [22]:
idx = random.randint(0, len(train))
idx

37754

In [25]:
print(train.tokens[idx])
print(train.tags[idx])
print(len(train.tokens[idx]))
print(len(train.tags[idx]))

['بالنسبة', 'للطلاب', 'ذوي', 'الإعاقة', 'البصرية', '،', 'يتم', 'توفير', 'خدمات', 'خاصة', 'من', 'قبل', 'Paradigm', '.', 'يرجى', 'الإشارة', 'إلى', 'ذلك', 'في', '02', '/', '09', '/', '1904', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DOB', 'I-DOB', 'I-DOB', 'I-DOB', 'I-DOB', 'O']
192
145


##### Convert tokens and tags columns to lists

In [26]:
import ast

In [38]:
print(ast.literal_eval(train.tokens[idx]))
print(type(ast.literal_eval(train.tokens[idx])), len(ast.literal_eval(train.tokens[idx])))
print(ast.literal_eval(train.tags[idx]))
print(type(ast.literal_eval(train.tags[idx])), len(ast.literal_eval(train.tags[idx])))

['بالنسبة', 'للطلاب', 'ذوي', 'الإعاقة', 'البصرية', '،', 'يتم', 'توفير', 'خدمات', 'خاصة', 'من', 'قبل', 'Paradigm', '.', 'يرجى', 'الإشارة', 'إلى', 'ذلك', 'في', '02', '/', '09', '/', '1904', '.']
<class 'list'> 25
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DOB', 'I-DOB', 'I-DOB', 'I-DOB', 'I-DOB', 'O']
<class 'list'> 25


In [40]:
tokens = train["tokens"].apply(ast.literal_eval)
tokens

,tokens
0,"[مرحبا, Johan, Walker, ،, إشعار, الامتثال, :, ..."
1,"[نحن, نقوم, بمراجعة, القضية, التي, رفعها, كادي..."
2,"[بسبب, اتفاقية, السرية, ،, لا, يمكن, للمريضة, ..."
3,"[عزيزي, ولي, الأمر, ،, يلزم, الحصول, على, إذن,..."
4,"[تنظيم, اجتماع, حول, 7, سنوات, (, Genderqueer,..."
...,...
209256,"[عزيزتي, الأنثى, ،, حدث, التعليم, الفني, في, ا..."
209257,"[هل, يمكننا, ترتيب, موعد, مع, والدي, نيكو, ؟, ..."
209258,"[يرجى, تأكيد, استلام, الزي, الجديد, عن, طريق, ..."
209259,"[أظهر, الشخص, ذو, العيون, ذات, اللون, البني, و..."


In [42]:
tags = train['tags'].apply(ast.literal_eval)
tags

,tags
0,"[O, B-FIRSTNAME, B-LASTNAME, O, O, O, O, O, O,..."
1,"[O, O, O, O, O, O, B-FIRSTNAME, B-LASTNAME, O,..."
2,"[O, O, O, O, O, O, O, B-SEX, O, O, O, B-PREFIX..."
3,"[O, O, O, O, O, O, O, O, O, B-DATE, I-DATE, I-..."
4,"[O, O, O, B-AGE, I-AGE, O, B-GENDER, O, O, O, ..."
...,...
209256,"[O, B-SEX, O, O, O, O, O, B-SECONDARYADDRESS, ..."
209257,"[O, O, O, O, O, O, B-MIDDLENAME, O, O, O, O, O..."
209258,"[O, O, O, O, O, O, O, O, O, O, B-EMAIL, I-EMAI..."
209259,"[O, O, O, O, O, O, O, O, B-HEIGHT, I-HEIGHT, I..."


In [44]:
train.tokens, train.tags = tokens, tags

In [ ]:
length_comparison_result = train.apply(lambda row: len(train.tokens) == len(train.tags), axis=1)
print(f"\nNumber of rows with mismatching lengths: {sum(~length_comparison_result)}")

##### Use only the needed tags

In [78]:
tags_to_mask =  [
 'EMAIL',
 'IBAN',
 'PERSONNAME',
 'IPV6',
 'MASKEDNUMBER',
 'CREDITCARDNUMBER',
 'LITECOINADDRESS',
 'BIC',
 'PHONENUMBER',
 'ORDINALDIRECTION',
 'VEHICLEVRM',
 'CREDITCARDINUMBER',
 'USERNAME',
 'IPV4',
 'PASSWORD',
 'MAC',
 'BITCOINADDRESS',
 'SECONDARYADDRESS',
 'CREDITCARDCVV',
 'PHONEIMEI',
 'ACCOUNTNAME',
 'BUILDINGNUMBER',
 'VEHICLEVIN',
 'STREET',
 'URL',
 'IP',
 'ZIPCODE',
 'ACCOUNTNUMBER',
 'FIRSTNAME',
 'LASTNAME',
 'NEARBYGPSCOORDINATE',
 'ETHEREUMADDRESS',
 'PIN',
 'SSN']

In [79]:
def mask_tags(tags_list, tags_to_mask):
  """Masks specified tags in a list.

  Args:
    tags_list: A list of tags.
    tags_to_mask: A list of tags to be masked.

  Returns:
    A new list with the specified tags masked.
  """
  masked_tags = []
  for tag in tags_list:
    if tag[2:] in tags_to_mask:
      masked_tags.append("[MASKED]")
    else:
      masked_tags.append("O")
  return masked_tags

In [80]:
train['masked_tags'] = train['tags'].apply(lambda x: mask_tags(x, tags_to_mask))

In [81]:
train['masked_tags'][5:11]

,masked_tags
5,"[O, [MASKED], O, O, O, O, O, O, O, O, O, O, [M..."
6,"[O, O, O, O, O, O, O, O, O, O, O, O, [MASKED],..."
7,"[O, [MASKED], O, O, O, O, O, O, O, O, O, O, O,..."
8,"[O, O, O, O, O, O, O, O, [MASKED], [MASKED], O..."
9,"[O, O, O, O, O, O, O, O, O, [MASKED], O, O, O,..."
10,"[O, O, O, O, O, O, O, [MASKED], [MASKED], [MAS..."


In [82]:
print(train['masked_tags'][0])
print(train['tokens'][0])
print(train['tags'][0])

['O', '[MASKED]', '[MASKED]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[MASKED]', '[MASKED]', 'O', 'O', 'O', 'O', 'O']
['مرحبا', 'Johan', 'Walker', '،', 'إشعار', 'الامتثال', ':', 'تتطلب', 'التغييرات', 'الأخيرة', 'في', 'التشريعات', 'في', 'سانت', 'غالن', 'منك', 'تحديث', 'Suite', '559', 'الخاص', 'بك', 'في', 'سجلاتنا', '.']
['O', 'B-FIRSTNAME', 'B-LASTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-STATE', 'I-STATE', 'O', 'O', 'B-SECONDARYADDRESS', 'I-SECONDARYADDRESS', 'O', 'O', 'O', 'O', 'O']


In [83]:
length_comparison_result = train.apply(lambda row: len(train.tokens) == len(train.masked_tags), axis=1)
print(f"\nNumber of rows with mismatching lengths: {sum(~length_comparison_result)}")


Number of rows with mismatching lengths: 0


##### Creating new masked sentences

In [ ]:
def create_masked_sentence(row):
  """Creates a masked sentence from tokens and masked tags.

  Args:
    row: A pandas DataFrame row with 'tokens' and 'masked_tags' columns.

  Returns:
    A string representing the masked sentence.
  """
  masked_sentence_tokens = []
  for token, tag in zip(row['tokens'], row['masked_tags']):
    if tag == '[MASKED]':
      masked_sentence_tokens.append('[masked]')
    else:
      masked_sentence_tokens.append(token)
  return ' '.join(masked_sentence_tokens)

In [ ]:
# Apply the function to each row to create the new masked sentence column
train['masked_sentence'] = train.apply(create_masked_sentence, axis=1)

# To see the result, you can display the masked_sentence column
print(train['masked_sentence'].head())

# Loading Model

### SILMA-2B-4bit

In [ ]:
# !pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
model_id = "silma-ai/SILMA-Kashif-2B-Instruct-v1.0-bnb-4bit" # Quantized version
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config = quantization_config,
)

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
messages = [
    {"role": "user", "content": "كم عدد أيام الأسبوع؟"},
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

input_ids

{'input_ids': tensor([[     2,    106,   1645,    108,  26813,  56884, 155468,  47324, 112339,
         236391,    107,    108]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
outputs = model.generate(**input_ids, max_new_tokens=256)

In [ ]:
tokenizer.decode(outputs[0]).split("<start_of_turn>")[-1]

'\nكم عدد أيام الأسبوع؟<end_of_turn>\n\n\nيجب أن يتم حساب عدد أيام الأسبوع من خلال حساب عدد أيام الأسبوع في الأسبوع المحدد.\n\nيجب أن يتم حساب عدد أيام الأسبوع من خلال حساب عدد أيام الأسبوع في الأسبوع المحدد.\n\nيجب أن يتم حساب عدد أيام الأسبوع من خلال حساب عدد أيام الأسبوع في الأسبوع المحدد.\n\nيجب أن يتم حساب عدد أيام الأسبوع من خلال حساب عدد أيام الأسبوع في الأسبوع المحدد.\n\nيجب أن يتم حساب عدد أيام الأسبوع من خلال حساب عدد أيام الأسبوع في الأسبوع المحدد.\n\nيجب أن يتم حساب عدد أيام الأسبوع من خلال حساب عدد أيام الأسبوع في الأسبوع المحدد.\n\nيجب أن يتم حساب عدد أيام الأسبوع من خلال حساب عدد أيام الأسبوع في الأسبوع المحدد.\n\nيجب أن يتم حساب عدد أيام الأسبوع من خلال حساب عدد أيام الأسبوع في الأسبوع المحدد.\n\nيجب أن يتم حساب عدد أيام الأسبوع من خلال حساب عدد أيام الأسبوع في الأسبوع المحدد.\n\nيجب أن يتم حساب عدد أيام الأسبوع من خلال حساب عدد أيام الأسبوع في الأسبوع المحدد.\n\nيجب أن يتم حساب عدد أيام الأسبوع من خلال حساب عدد أيام الأس'

### SILMA-2B

In [ ]:
import torch
from transformers import pipeline

In [ ]:
pipe = pipeline(
    "text-generation",
    model="silma-ai/SILMA-Kashif-2B-Instruct-v1.0",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda"
)

config.json:   0%|          | 0.00/974 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/47.1k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
messages = [
    {"role": "user",
     "content": """ Mask the PII as [masked] in the given source sentence

     source: I sent my email address M.helmy@nu.edu.eg to my supervisor dr. gahda khoriba in Nile University.

     target:
     """
     }
]

outputs = pipe(messages, max_new_tokens=600)
response = outputs[0]["generated_text"][-1]["content"].strip()

In [ ]:
response

'I sent my email address [masked] to my supervisor dr. gahda khoriba in Nile University.'

In [ ]:
messages = [
    {"role": "user",
     "content": f""" Mask the PII as [masked] in the given source sentence

     source: {df["source"][0]}

     target:
     """
     }
]

outputs = pipe(messages, max_new_tokens=600)
response = outputs[0]["generated_text"][-1]["content"].strip()

In [ ]:
print(df["source"][0])
print(response)

الأستاذ أدولفوس ريغان زييمان، بصفتك المدير التنفيذي الرئيسي لتطبيقات المركز في شركة ماكلوغلين، نادر وبوردي، معرفتك بإدارة التغيير مهمة لتحول شركتنا. نطلب منك إعداد استراتيجية لإدارة التغيير.
الأستاذ [masked] ريغان زييمان، بصفتك المدير التنفيذي الرئيسي لتطبيقات المركز في شركة [masked], نادر وبوردي، معرفتك بإدارة التغيير مهمة لتحول شركتنا. نطلب منك إعداد استراتيجية لإدارة التغيير.
